## Labeliser des documents

Un llm peut être utilisée afin de classifier (labéliser) un document. Ces labels peuvent être extrêmement divers:
* Sentiment
* Langage
* Style (formel, informel etc.)
* Sujets du document
* Tendance pollitique


La classification requiert deux composants:
* une `function` permettant de spécifier comment le modèle doit tagguer le document
* un `schema` définissant les labels souhaités

La `fonction` est assurée directement par Lanchain (via l'utilisation de `with_structured_output` sur le modèle).

Le `schema` lui est adapté selon notre besoin. 

Ici nous créons un premier schma `BasicClassification` afin d'extraire le sentiment, le ton, l'agressivité et le langue du document.



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
"""
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class BasicClassification(BaseModel):
    sentiment: str = Field(        
        ...,
        description="The sentiment of the text"),
    tone: str = Field(        
        ...,
        description="The tone of the text"),
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
    )
    language: str = Field(        
        ...,
        description="The language the text is written in")


# LLM
llm_basic_classification = ChatOpenAI(temperature=0, model="gpt-4o").with_structured_output(
BasicClassification
)

basic_tagging_chain = tagging_prompt | llm_basic_classification

In [ ]:
input = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
res = basic_tagging_chain.invoke({"input": input})
res.dict()

In [ ]:
input = "Weather is ok here, I can go outside without much more than a coat"
res = basic_tagging_chain.invoke({"input": input})
res.dict()

Pour améliorer les résultats, nous pouvons créons un nouveau schéma pour guider d'avanatge le modèle sur ce que l'on attend de lui.

In [ ]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    tone: str = Field(        
    ...,
    description="The tone of the text",
    enum=["formal", "informal", "neutral"]),
    language: str = Field(
        ...,
        description="The language the text is written in",
        enum=["spanish", "english", "french", "german", "italian"]
    )

basic_tagging_chain = tagging_prompt | llm_basic_classification

input = "Weather is ok here, I can go outside without much more than a coat"
res = basic_tagging_chain.invoke({"input": input})
res.dict()